# Fine-tuning ESM2 on promiscuous esterases

In [ ]:
from transformers import EsmTokenizer, EsmForMaskedLM, TrainingArguments, Trainer
import torch

# Choose the ESM2 model size (e.g., 'esm2_t33_650M_UR50D', 'esm2_t12_35M_UR50D')
# Larger models are more powerful but require more resources.
model_name = "facebook/esm2_t33_650M_UR50D"
tokenizer = EsmTokenizer.from_pretrained(model_name)
model = EsmForMaskedLM.from_pretrained(model_name)